In [1]:
pip install nptdms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for nptdms: filename=npTDMS-1.7.0-py3-none-any.whl size=104710 sha256=d8249b4915d9aafb5a574b0bc80de4b660e4d9d6b4224be2848cb58b9b9d0576
  Stored in directory: /root/.cache/pip/wheels/24/61/d6/bdb3426f132dc4ae7ecb0606257c0cb108f332025008d29750
Successfully built nptdms
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nptdms import TdmsFile


#for traning data Librares:
import argparse
import joblib
import os
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def read_tdms(file_path_key):
  
    tdms_file = TdmsFile.read(file_path_key)

    data = {}
    channels = []
    for group in tdms_file.groups():
        for channel in group.channels():
            channel_name = channel.name
            data[channel_name] = channel[:]
            channels.append(channel_name)
   
    return data, channels

def preprocessing():
    file_path_key1 = '/kaggle/input/tdms-datasets/Healthy_Q110_R3_05_16_2023_12_52_47.tdms'
    file_path_key2 = '/kaggle/input/tdms-datasets/Unhealthy_Q110_R5_05_16_2023_15_08_23.tdms'
    file_path_key3 = '/kaggle/input/tdms-datasets/Healthy_Q110_R18_05_16_2023_13_26_57.tdms'
    file_path_key4 = '/kaggle/input/tdms-datasets/Unhealthy_R4_05_24_2023_11_52_49.tdms'


    healthy_data_df1, healthy_channels_df1    = read_tdms(file_path_key1)
    unhealthy_data_df1, unhealthy_channels_df1 = read_tdms(file_path_key2)
    healthy_data_df2, healthy_channels_df2    = read_tdms(file_path_key3)
    unhealthy_data_df2, unhealthy_channels_df2 = read_tdms(file_path_key4)
    
    
    df1=pd.DataFrame(healthy_data_df1)
    df1['Status'] = [0] * len(df1)
    print("healthy_data frame1",df1.shape)
    
    df2=pd.DataFrame(unhealthy_data_df1)
    df2['Status'] = [1] * len(df2)
    print("Unhealthy_data_frame2", df2.shape)
    
    df3=pd.DataFrame(healthy_data_df2)
    df3['Status'] = [0] * len(df3)
    print("healthy_data_frame3",df3.shape)
    
    df4=pd.DataFrame(unhealthy_data_df2)
    df4['Status'] = [1] * len(df4)
    print("unhealthy_data_frame4", df4.shape)
    
    
    
    
    data = pd.concat([df1, df2, df3, df4], ignore_index=True)
    
    if 'Time' in data.columns:
        df = data.drop('Time', axis=1)
        
        
    # Check for missing values in the DataFrame
    missing_values = df.isnull().sum()
    # print(missing_values)

    df.drop(['IW4-0623-0-CHAN-4', 'P-0201-0-CHAN-3', 'P-0201-0-CHAN-4'], axis=1, inplace=True)

    df.dropna(axis=0, inplace=True)
    print("Printing Whole dataset shape \n", df.shape)
    
    
    
    desired_order = ['B7043_18A', 'B5395_18A', 'B5384_18A', 'B5408_18A', 'IW4-0626-0-CHAN-1', 'IW4-0626-0-CHAN-2', 'LV219701_20A', 'B7046_18A', 'B4524_18A', 'B7032_18A', 'B5401_18A', 'B7030_18A', 'IW4-0623-0-CHAN-1', 'A2149', 'B7650_19A', 'B7035_18A', 'B6178_18A', 'B5397_18A', 'B7031_18A', 'B4546_18A', 'IW4-0627-0-CHAN-1', 'IW4-0627-0-CHAN-2', 'A2146_18A', 'B7045_18A', 'LV4231', 'B7033_18A', 'B7056_18A', 'B7639_19A', 'B5412_18A', 'B7037_18A', 'B7062_19A', 'LV207519_18A', 'B5381_18A', 'B6182_18A', 'B5398_18A', 'B7645_19A', 'B7648_19A', 'A2148', 'B6192_18A', 'B7643_19A', 'B7051_18A', 'B7061_18A', 'B4526_18A', 'B7038_18A', 'B7057_18A', 'B4541_18A', 'B7036_18A', 'B7039_18A', 'B7048_18A', 'B7049_18A', 'B5411_18A', 'B5406_18A', 'B5404_18A', 'B4523_18A', 'IW4-0624-0-CHAN-3', 'IW4-0624-0-CHAN-4', 'B5410_18A', 'B7055_18A', 'B7054_18A', 'B6181_18A', 'B7029_18A', 'B4531_18A', 'B7044_18A', 'B5382_18A', 'Status']

    df= df.reindex(columns=desired_order)

    
    
    X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Status", axis=1), df["Status"], test_size=0.25, random_state=42
    )

    trainX = pd.DataFrame(X_train)
    trainX["Status"] = y_train

    testX = pd.DataFrame(X_test)
    testX["Status"] = y_test  
    
    
    
    trainX.to_csv("/kaggle/working/Bridge_train.csv")
    testX.to_csv("/kaggle/working/Bridge_test.csv")
    
    
    print("\n DONE... Pre processing completed successfully and Model is ready to be trained \n ")
    train_path  = "/kaggle/working/Bridge_train.csv"
    test_path   = "/kaggle/working/Bridge_test.csv"

    return train_path, test_path


if __name__ == "__main__":
    train_path, test_path = preprocessing()
    print("Traing my Model Started...\n ")
    print("extracting arguments \n")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify  we  use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default="/kaggle/working/")
    parser.add_argument("--train-path", type=str, default=train_path)
    parser.add_argument("--test-path", type=str, default=test_path)
    
    
    # Adding Feature and Target Columns:
    features= 'B7043_18A B5395_18A B5384_18A B5408_18A IW4-0626-0-CHAN-1 IW4-0626-0-CHAN-2 LV219701_20A B7046_18A B4524_18A B7032_18A B5401_18A B7030_18A IW4-0623-0-CHAN-1 A2149 B7650_19A B7035_18A B6178_18A B5397_18A B7031_18A B4546_18A IW4-0627-0-CHAN-1 IW4-0627-0-CHAN-2 A2146_18A B7045_18A LV4231 B7033_18A B7056_18A B7639_19A B5412_18A B7037_18A B7062_19A LV207519_18A B5381_18A B6182_18A B5398_18A B7645_19A B7648_19A A2148 B6192_18A B7643_19A B7051_18A B7061_18A B4526_18A B7038_18A B7057_18A B4541_18A B7036_18A B7039_18A B7048_18A B7049_18A B5411_18A B5406_18A B5404_18A B4523_18A IW4-0624-0-CHAN-3 IW4-0624-0-CHAN-4 B5410_18A B7055_18A B7054_18A B6181_18A B7029_18A B4531_18A B7044_18A B5382_18A'
    target ='Status'
    
    
    parser.add_argument(
        "--features", type=str , default=features
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str , default= target
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("........Reading Data from CSV files........")
    
    train_df = pd.read_csv(args.train_path)
    test_df = pd.read_csv(args.test_path)

    print("......Building Training and Testing Datasets....\n ")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print("....Using RandomForestClassifier Algorithm and Fitting the Model....\n ")
    model = RandomForestClassifier(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    print("Printing Classification Report \n ",classification_report(y_test, y_pred))
    
    
    # print abs error
    print("Trying to get the absolute Value at 10, 50, 90 percentile \n ")
    abs_err = np.abs(model.predict(X_test) - y_test)
    
    
    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("Model is created at " + path)
    # print("min_samples_leaf is ", args.min_samples_leaf)

healthy_data frame1 (12455, 66)
Unhealthy_data_frame2 (3667, 66)
healthy_data_frame3 (3147, 66)
unhealthy_data_frame4 (7584, 66)
Printing Whole dataset shape 
 (19269, 65)

 DONE... Pre processing completed successfully and Model is ready to be trained 
 
Traing my Model Started...
 
extracting arguments 

........Reading Data from CSV files........
......Building Training and Testing Datasets....
 
....Using RandomForestClassifier Algorithm and Fitting the Model....
 
Printing Classification Report 
                precision    recall  f1-score   support

           0       1.00      1.00      1.00      3885
           1       1.00      1.00      1.00       933

    accuracy                           1.00      4818
   macro avg       1.00      1.00      1.00      4818
weighted avg       1.00      1.00      1.00      4818

Trying to get the absolute Value at 10, 50, 90 percentile 
 
AE-at-10th-percentile: 0.0
AE-at-50th-percentile: 0.0
AE-at-90th-percentile: 0.0
Model is created at /ka